In [1]:
import pandas as pd
import numpy as np
import os
import mzbatch
from sklearn.metrics.pairwise import cosine_similarity
import importlib as imp
imp.reload(mzbatch)
import subprocess
import warnings
warnings.filterwarnings("ignore") 

In [2]:
root_path = '../../Fulaoshi/230821_mzML/'
middle_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
out_root_path = '../../Fulaoshi/230821_mzML_res/rerun240410'

In [ ]:
"""Reference"""
ref_all_df = pd.read_excel("../../Fulaoshi/230821_mzML/ref_all-revise20240626_处理只有13个能量.xlsx",sheet_name="concat",index_col=0)
ref_all_df['batch_sampleid'] = ref_all_df['batch']+'/'+ref_all_df['sampleid'].astype(str)
ref_all_df['bscp'] = ref_all_df['batch']+'+'+ref_all_df['sampleid'].astype(str)+'+'+ref_all_df['CAS No.']+'+'+ref_all_df['pos_neg']
ref_all_df.head()


,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,Adduct,batch_sampleid,ev_nums,ev_list
1104,873_C18,C18,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.11960,285.0,133000000,873_C18+26386+1867-73-8+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1106,873_C18,C18,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.22160,874.8,111000000,873_C18+26386+50-22-6+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1108,873_C18,C18,26386,Hexylresorcinol,136-77-6,C12H18O2,194.27,pos,1,195.13790,936.0,994000,873_C18+26386+136-77-6+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1110,873_C18,C18,26386,"3,4-Dicaffeoylquinic acid",14534-61-3,C25H24O12,516.45,pos,1,517.13405,643.2,1010000,873_C18+26386+14534-61-3+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1112,873_C18,C18,26386,6-Benzylaminopurine,1214-39-7,C12H11N5,225.25,pos,1,226.10870,599.4,375000000,873_C18+26386+1214-39-7+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."


In [1812]:
ref_all_df.shape

(13354, 17)

In [1666]:
bscp_list = ref_all_df['bscp'].unique().tolist()
len(bscp_list)

13354

In [1667]:
""" Add ev_nums and ev_list for ref_all """
ref_all_df["ev_nums"]=0
ref_all_df["ev_list"]=""
for bscp in bscp_list:
    temp_con1_df = con1_info_df[(con1_info_df["bscp"]==bscp) & (con1_info_df["EV"]!=204060)]
    ev_list = temp_con1_df["EV"].unique().tolist()
    ev_list.sort()
    ev_nums = len(ev_list)
    ref_mask = (ref_all_df["bscp"] == bscp)
    ref_all_df.loc[ref_mask,["ev_nums","ev_list"]] = ev_nums,str(ev_list)
    # break


In [ ]:
ref_all_df.to_csv("../../Fulaoshi/230821_mzML/ref_all_add_evlist_and_nums.csv")

In [ ]:
ref_all_df = pd.read_csv("../../Fulaoshi/230821_mzML/ref_all_add_evlist_and_nums.csv",index_col=0)

In [6]:
ref_all_df.shape

(13354, 17)

In [7]:
ref_all_df.head()

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,Adduct,batch_sampleid,ev_nums,ev_list
1104,873_C18,C18,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.11960,285.0,133000000,873_C18+26386+1867-73-8+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1106,873_C18,C18,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.22160,874.8,111000000,873_C18+26386+50-22-6+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1108,873_C18,C18,26386,Hexylresorcinol,136-77-6,C12H18O2,194.27,pos,1,195.13790,936.0,994000,873_C18+26386+136-77-6+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1110,873_C18,C18,26386,"3,4-Dicaffeoylquinic acid",14534-61-3,C25H24O12,516.45,pos,1,517.13405,643.2,1010000,873_C18+26386+14534-61-3+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1112,873_C18,C18,26386,6-Benzylaminopurine,1214-39-7,C12H11N5,225.25,pos,1,226.10870,599.4,375000000,873_C18+26386+1214-39-7+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."


In [8]:
con1_info_df = pd.read_excel(f'{out_root_path}/con1_info_df.xlsx',index_col=0)
con1_info_df['bscp']=con1_info_df['batch']+'+'+con1_info_df['sampleid'].astype(str)+'+'+con1_info_df['CAS No.']+'+'+con1_info_df['pos_neg']
con1_info_df.head()

,batch,sampleid,CAS No.,EV,pos_neg,mz,Product Name,filename,refer_valid,ref_rt,...,mol_intensity,is_max_base,base_intensity,ms1_base_rate,ms1_mol_rate,bscpe,is_max_ms1_39,filepath,filter_filepath,bscp
5,700_Amide,S98,576-42-1,1,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE1_neg_MS2_12266_818.46.csv,1,826.2,...,1321181.90,0,1321181.90,1.372508,1.372508e+00,700_Amide+S98+576-42-1+neg+1,1,700_Amide/NCE_compound/S98/576-42-1_NCE1_neg/5...,700_Amide/NCE_compound_filter/S98/576-42-1_NCE...,700_Amide+S98+576-42-1+neg
18,700_Amide,S98,576-42-1,10,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE10_neg_MS2_12301_818.49.csv,1,826.2,...,1019900.40,0,1019900.40,1.691987,1.691987e+00,700_Amide+S98+576-42-1+neg+10,1,700_Amide/NCE_compound/S98/576-42-1_NCE10_neg/...,700_Amide/NCE_compound_filter/S98/576-42-1_NCE...,700_Amide+S98+576-42-1+neg
31,700_Amide,S98,576-42-1,100,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE100_neg_MS2_12241_814.59.csv,1,826.2,...,-1.00,0,81837.15,17.406986,-1.424538e+06,700_Amide+S98+576-42-1+neg+100,1,700_Amide/NCE_compound/S98/576-42-1_NCE100_neg...,700_Amide/NCE_compound_filter/S98/576-42-1_NCE...,700_Amide+S98+576-42-1+neg
42,700_Amide,S98,576-42-1,15,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE15_neg_MS2_12171_815.44.csv,1,826.2,...,685409.44,0,685409.44,2.262262,2.262262e+00,700_Amide+S98+576-42-1+neg+15,1,700_Amide/NCE_compound/S98/576-42-1_NCE15_neg/...,700_Amide/NCE_compound_filter/S98/576-42-1_NCE...,700_Amide+S98+576-42-1+neg
57,700_Amide,S98,576-42-1,20,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE20_neg_MS2_12266_818.88.csv,1,826.2,...,432622.44,0,432622.44,3.678289,3.678289e+00,700_Amide+S98+576-42-1+neg+20,1,700_Amide/NCE_compound/S98/576-42-1_NCE20_neg/...,700_Amide/NCE_compound_filter/S98/576-42-1_NCE...,700_Amide+S98+576-42-1+neg


In [9]:
con1_info_df["bscpe"] = con1_info_df['batch']+'+'+con1_info_df['sampleid'].astype(str)+'+'+con1_info_df['CAS No.']+'+'+con1_info_df['pos_neg']+'+'+con1_info_df['EV'].astype(str)
bscpe_list = con1_info_df["bscpe"].unique().tolist()
len(bscpe_list), bscpe_list[0:5]

(190786,
 ['700_Amide+S98+576-42-1+neg+1',
  '700_Amide+S98+576-42-1+neg+10',
  '700_Amide+S98+576-42-1+neg+100',
  '700_Amide+S98+576-42-1+neg+15',
  '700_Amide+S98+576-42-1+neg+20'])

In [1593]:
con1_info_df["sepuzhu"] = con1_info_df["batch"].apply(lambda x:x.split("_")[1])

In [ ]:
# Mode list style sepuzhu+pos_neg+EV
con1_info_df["spe"] = con1_info_df['sepuzhu'].astype(str)+'+'+con1_info_df['pos_neg']+'+'+con1_info_df['EV'].astype(str)
spe_list = con1_info_df["spe"].unique().tolist()
len(spe_list), spe_list[0:5]


(56,
 ['Amide+neg+1',
  'Amide+neg+10',
  'Amide+neg+100',
  'Amide+neg+15',
  'Amide+neg+20'])

# jaccard similarity

In [859]:
import numpy as np

def fu_jaccard_similarity_vector(P, Q):
    dot_product = np.dot(P, Q)
    sum_of_squares_p = np.dot(P, P)
    sum_of_squares_q = np.dot(Q, Q)
    
    jaccard_index = dot_product / (sum_of_squares_p + sum_of_squares_q - dot_product)
    return jaccard_index

# Example vectors
P = np.array([1, 2, 3])
Q = np.array([1.1, 2.2, 3.3])

# Calculate Jaccard similarity
similarity = fu_jaccard_similarity(P, Q)
print(f"Jaccard Similarity: {similarity}")


Jaccard Similarity: 0.9909909909909907


In [860]:
import numpy as np

def fu_jaccard_similarity_matrix(X, Y=None):
    # If Y is None, use X for both arguments (i.e., compare all pairs within X)
    if Y is None:
        Y = X

    # Ensure input is numpy arrays
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)

    # Compute dot products
    dot_product = np.dot(X, Y.T)
    
    # Compute the sum of squares for each vector
    sum_of_squares_X = np.sum(X ** 2, axis=1)[:, np.newaxis]  # Make column vector
    sum_of_squares_Y = np.sum(Y ** 2, axis=1)  # Make row vector
    
    # Compute the Jaccard similarity matrix
    denominator = sum_of_squares_X + sum_of_squares_Y - dot_product
    jaccard_index = dot_product / denominator
    
    return jaccard_index

# Example matrices
X = np.array([[1, 2, 3], [4, 5, 6]])
Y = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Calculate Jaccard similarity
similarity_matrix = fu_jaccard_similarity_matrix(X)
print("Jaccard Similarity Matrix:\n", similarity_matrix)


Jaccard Similarity Matrix:
 [[1.         0.54237288]
 [0.54237288 1.        ]]


# Establish isomers, (positive and negative, different modes) references

In [1827]:
isomeride_df = pd.DataFrame(columns=["sepuzhu", "pos_neg", "bscp1", "b1_name", "b1_adduct",
                                     "bscp2_list", "b2_name_list", "b2_adduct_list"])

In [1828]:
pattern_list = [["Amide","pos"], ["Amide","neg"], ["C18","pos"], ["C18","neg"]]
ev14_list = [1,10,100,15,20,25,30,35,40,50,60,70,80,204060]
# ev_list = ['1', '10', '100', '15', '20', '25', '30', '35', '40', '50', '60','70', '80']

In [2]:
"""Isomers"""
n=0
for [sepuzhu, pos_neg] in pattern_list:
    ref_pattern_df = ref_all_df[(ref_all_df["sepuzhu"]==sepuzhu) & (ref_all_df["pos_neg"]==pos_neg)]
    for i in ref_pattern_df.index:
        bscp1, mz, b1_adduct, b1_name = ref_pattern_df.loc[i,["bscp", "m/z", "Adduct", "Product Name"]]
        ref_isomeride_df = ref_pattern_df[ref_pattern_df["m/z"]==mz]
        if ref_isomeride_df.shape[0]>1:
            bscp2_list = ref_isomeride_df["bscp"].tolist()
            bscp2_list.remove(bscp1)
            b2_adduct_list = ref_isomeride_df["Adduct"].tolist()
            b2_adduct_list.remove(b1_adduct)
            b2_name_list = ref_isomeride_df["Product Name"].tolist()
            b2_name_list.remove(b1_name)
            
            isomeride_df.loc[n] = [sepuzhu, pos_neg, bscp1, b1_name, b1_adduct, 
                                   bscp2_list, b2_name_list, b2_adduct_list]
            n+=1
        # break


In [1833]:
isomeride_df["class"]="isomeride"
isomeride_df.head(2)

,sepuzhu,pos_neg,bscp1,b1_name,b1_adduct,bscp2_list,b2_name_list,b2_adduct_list,class
0,Amide,pos,BC_Amide+S109+14534-61-3+pos,"3,4-Dicaffeoylquinic acid",[M+H]+,[873_Amide+2_2+2450-53-5+pos],[Isochlorogenic acid A],[[M+H]+],isomeride
1,Amide,pos,873_Amide+26386+56-12-2+pos,γ-Aminobutyric acid,[M+H]+,"[873_Amide+26396+4249-19-8+pos, 873_Amide+2_1+...","[(S)-b-aminoisobutyric acid, NSC 16590, 3-Amin...","[[M+H]+, [M+H]+, [M+H]+, [M+H]+, [M+H]+, [M+H]+]",isomeride


In [ ]:
# Output to file
isomeride_df.to_csv(f"{out_root_path}/isomeride_df.csv")


In [1839]:
"""different patterns"""
def change_pattern(b1_sepuzhu):
    if b1_sepuzhu == "Amide":
        return "C18"
    elif b1_sepuzhu == "C18":
        return "Amide"

In [1928]:
diff_pattern_df = pd.DataFrame(columns=["sepuzhu", "pos_neg", "bscp1", "b1_name", "b1_adduct",
                                        "bscp2_list", "b2_name_list", "b2_adduct_list"])

In [1841]:
m=0
for i in ref_all_df.index:
    b1_cols_list = ['sepuzhu','Product Name', 'CAS No.', 'pos_neg', 'bscp', 'Adduct']
    # b2_cols_list = ['sepuzhu', 'pos_neg', 'bscp', 'Adduct']
    b1_sepuzhu, b1_name, b1_cas, b1_pos_neg, bscp1, b1_adduct = ref_all_df.loc[i,b1_cols_list]
    b2_sepuzhu = change_pattern(b1_sepuzhu)
    b2_temp_df = ref_all_df[(ref_all_df["sepuzhu"]==b2_sepuzhu) 
                            & (ref_all_df["CAS No."]==b1_cas) 
                            & (ref_all_df["pos_neg"]==b1_pos_neg)]
    if b2_temp_df.shape[0]==0:
        continue
    # bscp2_list = 
    bscp2_list = b2_temp_df["bscp"].tolist()
    b2_adduct_list = b2_temp_df["Adduct"].tolist()
    b2_name_list = b2_temp_df["Product Name"].tolist()

    diff_pattern_df.loc[m] = [b1_sepuzhu, b1_pos_neg, bscp1, b1_name, b1_adduct,
                              bscp2_list, b2_name_list, b2_adduct_list]
    m+=1
    # break

In [1842]:
diff_pattern_df["class"] = "diff_pattern"

In [1843]:
diff_pattern_df.head(2)

,sepuzhu,pos_neg,bscp1,b1_name,b1_adduct,bscp2_list,b2_name_list,b2_adduct_list,class
0,C18,pos,873_C18+26386+1867-73-8+pos,N6-Methyladenosine,[M+H]+,[873_Amide+26386+1867-73-8+pos],[N6-Methyladenosine],[[M+H]+],diff_pattern
1,C18,pos,873_C18+26386+50-22-6+pos,Corticosterone,[M+H]+,[873_Amide+26386+50-22-6+pos],[Corticosterone],[[M+H]+],diff_pattern


In [1846]:
diff_pattern_df.shape

(11604, 9)

In [1929]:
"""Merge isomeride_df和diff_pattern_df"""
con_isomeride_df = pd.concat([isomeride_df,diff_pattern_df])
con_isomeride_df.reset_index(inplace=True,drop=True)
con_isomeride_df.tail(2)

,sepuzhu,pos_neg,bscp1,b1_name,b1_adduct,bscp2_list,b2_name_list,b2_adduct_list,class
5064,C18,neg,700_C18+S104+28954-12-3+neg,L-Allothreonine,[M-H]-,"[873_C18+26391+672-15-1+neg, 873_C18+26396+72-...","[L-Homoserine, L-Threonine, D-Allothreonine]","[[M-H]-, [M-H]-, [M-H]-]",isomeride
5065,C18,neg,700_C18+S104+7048-04-6+neg,L-Cysteine hydrochloride monohydrate,[M-H]-,[873_C18+26397+52-90-4+neg],[L-Cysteine],[[M-H]-],isomeride


In [1933]:
con_isomeride_df.shape

(5066, 9)

In [ ]:
# Output to file
con_isomeride_df.to_csv(f"{out_root_path}/con_isomeride_df.csv")

In [13]:
con_isomeride_df = pd.read_csv(f"{out_root_path}/con_isomeride_df.csv")

In [1854]:
print(con1_info_df.columns)
con1_info_df.head(1)

Index(['batch', 'sampleid', 'CAS No.', 'EV', 'pos_neg', 'mz', 'Product Name',
       'filename', 'refer_valid', 'ref_rt', 'ref_intensity', 'is_max_ms1',
       'ms1_intensity', 'is_max_mol', 'mol_intensity', 'is_max_base',
       'base_intensity', 'ms1_base_rate', 'ms1_mol_rate', 'bscpe',
       'is_max_ms1_39', 'filepath', 'filter_filepath', 'bscp', 'sepuzhu',
       'spe'],
      dtype='object')


,batch,sampleid,CAS No.,EV,pos_neg,mz,Product Name,filename,refer_valid,ref_rt,...,base_intensity,ms1_base_rate,ms1_mol_rate,bscpe,is_max_ms1_39,filepath,filter_filepath,bscp,sepuzhu,spe
5,700_Amide,S98,576-42-1,1,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE1_neg_MS2_12266_818.46.csv,1,826.2,...,1321181.9,1.372508,1.372508,700_Amide+S98+576-42-1+neg+1,1,700_Amide/NCE_compound/S98/576-42-1_NCE1_neg/5...,700_Amide/NCE_compound_filter/S98/576-42-1_NCE...,700_Amide+S98+576-42-1+neg,Amide,Amide+neg+1


# cross-sectional similarity calculation

In [ ]:
"""Calculate the similarity between bscp1 and each element in bscp2_list"""
sim_method_list = ["cosine", "jaccard", "entropy"]
for sim_method in sim_method_list[1:2]:
    # Iterate through each row
    for i in con_isomeride_df.index[2488:2489]:
        if i % 100 == 0:
            print("==", i)
        sepuzhu, pos_neg, bscp1, b1_name, b1_adduct, bscp2_list, b2_name_list, b2_adduct_list, class_ = con_isomeride_df.loc[i, :]
        batch, sampleid, cas, pos_neg = bscp1.split("+")
        temp_bscp_list = [bscp1] + bscp2_list
        # Iterate through each energy level
        for ev in ev14_list:
            # print("111")
            temp_bscpe_list = [f"{bscp}+{ev}" for bscp in temp_bscp_list]

            bscpe_ms2_list = []
            for bscpe in temp_bscpe_list:
                temp_con1_info_df = con1_info_df[con1_info_df["bscpe"] == bscpe]
                if temp_con1_info_df.shape[0] > 0:
                    temp_filepath, theo_mz = temp_con1_info_df.loc[temp_con1_info_df.index[0], ["filter_filepath", "mz"]]
                    temp_bscpe = mzbatch.read_data(f"{out_root_path}/{temp_filepath}", theo_mz, rate=100)
                    bscpe_ms2_list.append(temp_bscpe)
                else:
                    # print(f"With {temp_bscpe_list[0]} as the reference, the corresponding {bscpe} was not found")
                    bscpe_ms2_list.append(pd.DataFrame({"Intensity": 0}, index=[-1]).rename_axis('Mass'))
            temp_bscpe_df = pd.concat(bscpe_ms2_list, axis=1)
            temp_bscpe_df.columns = [-1] + list(range(len(bscpe_ms2_list) - 1))
            temp_bscpe_df = temp_bscpe_df.fillna(0)
            temp_bscpe_df = mzbatch.to_relative_abundance(temp_bscpe_df)  # 240805 convert to relative abundance!! Very important
            pos_sim_list = mzbatch.cal_pos_sim(temp_bscpe_df, sim_method=sim_method)

            # Non-zero count for each column
            no0_nums_list = temp_bscpe_df.ne(0).sum(axis=0).tolist()
            bscp1_no0_nums_list = no0_nums_list[0]
            bscp2_no0_nums_list = no0_nums_list[1:]
            # Generate output file
            out_folder = f"{out_root_path}/{batch}/NCE_similar_{sim_method}/{sampleid}/"
            if not os.path.exists(out_folder):
                os.makedirs(out_folder)
                while os.path.exists(out_folder):
                    break
            out_filename = f"{temp_bscpe_list[0][temp_bscpe_list[0].index('+', temp_bscpe_list[0].index('+') + 1) + 1:]}"
            pd.DataFrame({"sepuzhu": sepuzhu, "pos_neg": pos_neg, "ev": ev, "class": class_,
                          "bscp1": bscp1, "b1_name": b1_name, "b1_adduct": b1_adduct,
                          "bscp2": bscp2_list, "b2_name": b2_name_list, "b2_adduct": b2_adduct_list,
                          "pos_sim": pos_sim_list, "square_pos_sim": np.square(pos_sim_list),
                          "bscp1_no0_nums_list": bscp1_no0_nums_list,
                          "bscp2_no0_nums_list": bscp2_no0_nums_list}).to_csv(f"{out_folder}/{out_filename}.csv")
            # print("Finished saving")

                # break
            # break
        # break1
    # break


## Merge files
- The two sim_methods are merged separately

In [17]:
batch_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
sim_method_list = ["cosine", "jaccard", "entropy"]

In [ ]:
# Merge single batch
# sim_method = sim_method_list[0]
# sim_method = sim_method_list[1]
sim_method = sim_method_list[2]
for batch in batch_list:
    # Result file
    similar_root_path = f'{out_root_path}/{batch}/NCE_similar_{sim_method}'
    similar_df_list = []
    
    for root, dirs, files in os.walk(similar_root_path):
        for similar_filename in files:
            one_similar_df = pd.read_csv(f'{root}/{similar_filename}', index_col=0)
            similar_df_list.append(one_similar_df)

    similar_all_df = pd.concat(similar_df_list)
    # similar_all_df.to_excel(f'{out_root_path}/{batch}/similar_all_df.xlsx')
    similar_all_df.to_csv(f'{out_root_path}/{batch}/similar_{sim_method}_all_df.csv')


In [ ]:
# Merge all batches
batch_similar_df_list = []

for batch in batch_list:
    batch_similar_df = pd.read_csv(f'{out_root_path}/{batch}/similar_{sim_method}_all_df.csv', index_col=0)
    batch_similar_df_list.append(batch_similar_df)
    
con_similar_df = pd.concat(batch_similar_df_list)
con_similar_df.to_csv(f'{out_root_path}/con_similar_{sim_method}_all_df.csv')


#Longitudinal calculation

In [1577]:
con_isomeride_df.shape

(17289, 9)

In [1633]:
""" Build standard_ms2_df, behavior mz, as energy """
def make_standard_ms2_df(temp_con1_info_df,out_res_path,temp_bscp):
    standard_ms2_list = []
    for standard_index in temp_con1_info_df.index:
        mz, filter_filepath = temp_con1_info_df.loc[standard_index,["mz", "filter_filepath"]]
        one_ms2_df = mzbatch.read_data(f'{out_root_path}/{filter_filepath}',mz,rate=100)
        standard_ms2_list.append(one_ms2_df)
    standard_ms2_df = pd.concat(standard_ms2_list,axis=1)
    standard_ms2_df.columns = temp_con1_info_df["EV"].tolist()
    standard_ms2_df = standard_ms2_df.fillna(0)
    standard_ms2_df = mzbatch.to_relative_abundance(standard_ms2_df)
    standard_ms2_df = standard_ms2_df.sort_index()
    ev_list = temp_con1_info_df["EV"].unique().tolist()
    ev_list.sort()
    standard_ms2_df = standard_ms2_df.loc[:,ev_list]
    standard_ms2_df.to_csv(f"{out_res_path}/{temp_bscp}.csv")

In [ ]:
""" A collection of 13 energies per compound is constructed according to the bscp. Output to NCE_merge13 with the file name bscp"""
for temp_bscp in bscp_list[1984:1985]:
    # temp_bscp = "700_C18+S101+56-73-5+pos"
    # temp_bscp = "873_Amide+26386+1477-68-5+neg" # No files found
    batch,sampleid,cas,pos_neg = temp_bscp.split("+")
    temp_con1_info_df = con1_info_df[(con1_info_df['bscp'] == temp_bscp) & (con1_info_df['EV'] != 204060)].copy()
    if temp_con1_info_df.shape[0]==0:
        print(temp_bscp, "No energy files found")
        continue
    out_res_path = f"{out_root_path}/{batch}/NCE_merge13/"
    if not os.path.exists(out_res_path):
        os.makedirs(out_res_path)
        while os.path.exists(out_res_path):
            break
    make_standard_ms2_df(temp_con1_info_df,out_res_path,temp_bscp)
    # break

In [1585]:
def change_cols(data_df, ev_intersection_list, col_pre):
    data_df = data_df.loc[:,ev_intersection_list]
    data_cols = [f"{col_pre}_{i}" for i in data_df.columns]
    data_df.columns = data_cols
    return data_df

In [1588]:
def read_bscp_merge13file(bscp):
    batch,sampleid,cas,pos_neg = bscp.split("+")
    filepath = f"{out_root_path}/{batch}/NCE_merge13/{bscp}.csv"
    # print(filepath)
    if not os.path.exists(filepath):
        return pd.DataFrame(columns=[-1])
    data_df = pd.read_csv(filepath,index_col=0)
    return data_df
a = read_bscp_merge13file(bscp1)
a.shape

(2, 13)

### Output NCE_align_cosine and NCE_align_res_cosine

In [ ]:
"""Generate column names"""
sim_method_list = ["cosine", "jaccard"]
top_colist = []
[top_colist.extend([f"top_{i}", f"top_{i}_sum", f"top_{i}_ev13"]) for i in range(1, 21)]
print(len(top_colist))
align_res_cols = ["sepuzhu", "pos_neg", "class", "bscp1", "b1_name", "b1_adduct", "b1_no0_linenums",
                  "bscp2", "b2_name", "b2_adduct", "b2_no0_linenums", "ev_nums", "ev_list", "is_same"] + top_colist
print(len(align_res_cols), align_res_cols)


60
74 ['sepuzhu', 'pos_neg', 'class', 'bscp1', 'b1_name', 'b1_adduct', 'b1_no0_linenums', 'bscp2', 'b2_name', 'b2_adduct', 'b2_no0_linenums', 'ev_nums', 'ev_list', 'is_same', 'top_1', 'top_1_sum', 'top_1_ev13', 'top_2', 'top_2_sum', 'top_2_ev13', 'top_3', 'top_3_sum', 'top_3_ev13', 'top_4', 'top_4_sum', 'top_4_ev13', 'top_5', 'top_5_sum', 'top_5_ev13', 'top_6', 'top_6_sum', 'top_6_ev13', 'top_7', 'top_7_sum', 'top_7_ev13', 'top_8', 'top_8_sum', 'top_8_ev13', 'top_9', 'top_9_sum', 'top_9_ev13', 'top_10', 'top_10_sum', 'top_10_ev13', 'top_11', 'top_11_sum', 'top_11_ev13', 'top_12', 'top_12_sum', 'top_12_ev13', 'top_13', 'top_13_sum', 'top_13_ev13', 'top_14', 'top_14_sum', 'top_14_ev13', 'top_15', 'top_15_sum', 'top_15_ev13', 'top_16', 'top_16_sum', 'top_16_ev13', 'top_17', 'top_17_sum', 'top_17_ev13', 'top_18', 'top_18_sum', 'top_18_ev13', 'top_19', 'top_19_sum', 'top_19_ev13', 'top_20', 'top_20_sum', 'top_20_ev13']


In [1954]:
ev_13_list = [1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 100]
ev_0160_list = [1, 10, 15, 20, 25, 30, 35, 40, 50, 60]
ev_0170_list = [1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70]
ev_2060_list = [20, 25, 30, 35, 40, 50, 60]
ev_204060_list = [20, 40, 60]
ev_select_dict = {"ev13":ev_13_list, "ev0160":ev_0160_list, "ev0170":ev_0170_list, "ev2060":ev_2060_list, "ev204060":ev_204060_list}
ev_select_name = "ev13"
ev_select_list = ev_select_dict[ev_select_name]
ev_select_list

[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 100]

In [1755]:
sim_method = sim_method_list[0]

for i in con_isomeride_df.index[0:]:
    sepuzhu, pos_neg, bscp1, b1_name, b1_adduct, bscp2_list, b2_name_list, b2_adduct_list, class_ = con_isomeride_df.loc[i,:]
    batch1, sampleid1, cas1, pos_neg1 = bscp1.split("+")
    bscp1_align_res_df = pd.DataFrame(columns=align_res_cols)
    n = 0
    for b2_index, bscp2 in enumerate(bscp2_list):
        """Merge b1 and b2"""
        batch2, sampleid2, cas2, pos_neg2 = bscp2.split("+")
        b2_adduct = b2_adduct_list[b2_index]
        b2_name = b2_name_list[b2_index]
    
        # Get energy intersection
        ref1_df = ref_all_df[ref_all_df["bscp"] == bscp1]
        ev1_nums, ev1_list = ref1_df.loc[ref1_df.index[0], ["ev_nums", "ev_list"]]
        if ev1_nums == 0:
            continue
        ref2_df = ref_all_df[ref_all_df["bscp"] == bscp2]
        ev2_nums, ev2_list = ref2_df.loc[ref2_df.index[0], ["ev_nums", "ev_list"]]
        if ev2_nums == 0:
            continue
        # Get intersection
        ev1_list = eval(ev1_list)
        ev2_list = eval(ev2_list)
        ev_intersection_list = list(set(ev1_list) & set(ev2_list) & set(ev_select_list))
        ev_intersection_list.sort()
        if len(ev_intersection_list) == 0:
            continue
        b1_cols = [f"b1_{i}" for i in ev_intersection_list]  # b1 energy columns
        b2_cols = [f"b2_{i}" for i in ev_intersection_list]  # b2 energy columns
            
        # Read files
        ev_intersection_list = [str(i) for i in ev_intersection_list]
        bscp1_df = read_bscp_merge13file(bscp1)
        bscp1_df = change_cols(bscp1_df, ev_intersection_list, "b1")

        bscp2_df = read_bscp_merge13file(bscp2)
        bscp2_df = change_cols(bscp2_df, ev_intersection_list, "b2")

        b12_df = pd.concat([bscp1_df, bscp2_df], axis=1)
        b12_df = b12_df.fillna(0)
        b12_df = b12_df.sort_index()
        if b12_df.shape[0] == 0:
            print(f"b1:{bscp1}, b2:{bscp2} is empty")
            continue
        
        # Add group column, group consecutive differences of 1 together
        b12_df['group'] = (b12_df.index.to_series().diff() != 1).cumsum()
        b12_bak_df = b12_df.copy(deep=True)
        # Take the maximum of each row
        b12_df = b12_df.groupby('group').agg(lambda x: x.max())
        # Remove the group column and set the smallest index value as the new index
        b12_df.index = b12_bak_df.groupby('group').apply(lambda x: x.index.min()).values
        
        # Calculate vertical similarity, do not add columns beforehand, otherwise it will cause errors
        if sim_method == "cosine":
            b12_df["cosine"] = np.square(np.diag(cosine_similarity(b12_df.loc[:, b1_cols].to_numpy(), b12_df.loc[:, b2_cols].to_numpy())))
        else:
            b12_df["jaccard"] = np.diag(mzbatch.fu_jaccard_similarity_matrix(b12_df.loc[:, b1_cols].to_numpy(), b12_df.loc[:, b2_cols].to_numpy()))
        align_out_path = f"{out_root_path}/{batch1}/NCE_align_{sim_method}_{ev_select_name}/{bscp1}"
        
        if not os.path.exists(align_out_path):
            os.makedirs(align_out_path)
            while os.path.exists(align_out_path):
                break
        # Calculate the sum intensity of b1 and b2
        b12_df["b1_sum_intensity"] = b12_df.loc[:, b1_cols].sum(axis=1)
        b12_df["b2_sum_intensity"] = b12_df.loc[:, b2_cols].sum(axis=1)
        b12_df["b12_sum_intensity"] = b12_df["b1_sum_intensity"] + b12_df["b2_sum_intensity"]
        b12_df.to_csv(f"{align_out_path}/{bscp2}.csv")
        
        """Output the results of vertical similarity statistics to NCE_align_res"""
        b12_no0_df = b12_df[~(b12_df[b1_cols] == 0).all(axis=1)]  # Filter rows where all 13 columns of b1 are not 0
        b12_no0_sort_df = b12_no0_df.sort_values(by="b12_sum_intensity", ascending=False)  # Sort by the sum intensity column, take top
        b1_no0_linenums = b12_no0_sort_df.shape[0]
        
        b2_no0_df = b12_df[~(b12_df[b2_cols] == 0).all(axis=1)]
        b2_no0_linenums = b2_no0_df.shape[0]

        is_same = (cas1 == cas2)
        top_res_list = []
        # Total only topn, max is 20
        len_topn = b12_no0_sort_df.shape[0] 
        if len_topn > 20:
            len_topn = 20
        for i in range(20):
            if i < len_topn:
                topi_index = b12_no0_sort_df.index[i]
                topi = b12_no0_sort_df.loc[topi_index, sim_method]
                topi_sum = [b12_no0_sort_df.loc[topi_index, "b1_sum_intensity"], b12_no0_sort_df.loc[topi_index, "b2_sum_intensity"]]
                topi_ev13 = [np.round(b12_no0_sort_df.loc[topi_index, b1_cols], 4).tolist(),
                             np.round(b12_no0_sort_df.loc[topi_index, b2_cols], 4).tolist()]
                top_res_list.extend([topi, topi_sum, topi_ev13])
            else:
                top_res_list.extend([-1, -1, -1])
        bscp1_align_res_df.loc[n] = [sepuzhu, pos_neg, class_, bscp1, b1_name, b1_adduct, b1_no0_linenums,
                                     bscp2, b2_name, b2_adduct, b2_no0_linenums,
                                     len(ev_intersection_list), ev_intersection_list, is_same] + top_res_list
        n += 1
        
    align_res_outpath = f"{out_root_path}/{batch1}/NCE_align_res_{sim_method}_{ev_select_name}/"
    if not os.path.exists(align_res_outpath):
        os.makedirs(align_res_outpath)
        while os.path.exists(align_res_outpath):
            break
    # if os.path.exists(f"{align_res_outpath}/{bscp1}.csv"):
    #     bscp1_align_res_df.to_csv(f"{align_res_outpath}/{bscp1}.csv", mode="a", header=False)
    # else:
    bscp1_align_res_df.to_csv(f"{align_res_outpath}/{bscp1}_{class_}.csv")
    
    # break

## Merge files

In [1983]:
batch_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
sim_method_list = ["cosine", "jaccard", "entropy"]

In [ ]:
for sim_method in sim_method_list:
    for ev_select in ev_select_dict.keys():
    # for ev_select in ["ev13"]:

        for batch in batch_list:
            # Result file
            align_res_outpath = f'{out_root_path}/{batch}/NCE_align_res_{sim_method}_{ev_select}'
            align_res_df_list = []

            for align_res_filename in os.listdir(align_res_outpath):
                one_align_res_df = pd.read_csv(f'{align_res_outpath}/{align_res_filename}', index_col=0)
                align_res_df_list.append(one_align_res_df)

            align_res_all_df = pd.concat(align_res_df_list)
            # similar_all_df.to_excel(f'{out_root_path}/{batch}/similar_all_df.xlsx')
            align_res_all_df.to_csv(f'{out_root_path}/{batch}/align_res_{sim_method}_{ev_select}_all_df.csv')


In [ ]:
for sim_method in sim_method_list:
    for ev_select in ev_select_dict.keys():
    # for ev_select in ["ev13"]:
        batch_align_res_df_list = []  # Modified on 0605
        for batch in batch_list:
            batch_align_res_df = pd.read_csv(f'{out_root_path}/{batch}/align_res_{sim_method}_{ev_select}_all_df.csv', index_col=0)
            batch_align_res_df_list.append(batch_align_res_df)

        con_align_res_df = pd.concat(batch_align_res_df_list)
        con_align_res_df.to_csv(f'{out_root_path}/con_align_res_{sim_method}_{ev_select}_all_df.csv')
